# Preamble

In [1]:
import os, sys
import pandas as pd
import numpy as np

In [2]:
sys.path.append('/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/code/func/')
from proj_environment import set_proj_env
from func import get_synth_cov

In [3]:
train_test_str = 'squeakycleanExclude'
exclude_str = 't1Exclude'
parc_str = 'schaefer'
parc_scale = 200
extra_str = ''
# extra_str = '_nuis-netdens'
# extra_str = '_nuis-str'
parcel_names, parcel_loc, drop_parcels, num_parcels, yeo_idx, yeo_labels = set_proj_env(train_test_str = train_test_str, exclude_str = exclude_str,
                                                                            parc_str = parc_str, parc_scale = parc_scale, extra_str = extra_str)

In [4]:
print(os.environ['MODELDIR_BASE'])
print(os.environ['MODELDIR'])

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_200_streamlineCount
/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_200_streamlineCount


## Load data

In [5]:
# Load data
df = pd.read_csv(os.path.join(os.environ['MODELDIR_BASE'], 'df_pheno.csv'))
df.set_index(['bblid', 'scanid'], inplace = True)

df_node = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_node_clean.csv'))
df_node.set_index(['bblid', 'scanid'], inplace = True)

# adjust sex to 0 and 1
df['sex_adj'] = df.sex - 1
print(df.shape)
print(df_node.shape)

(1080, 21)
(1080, 1001)


In [6]:
df.head()

,,squeakycleanExclude,ageAtScan1,ageAtScan1_Years,sex,race2,handednessv2,dti64MeanAbsRMS,dti64MeanRelRMS,dti64MaxAbsRMS,dti64MaxRelRMS,...,averageManualRating,Overall_Psychopathology,Psychosis_Positive,Psychosis_NegativeDisorg,AnxiousMisery,Externalizing,Fear,streamline_count,network_density,sex_adj
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,0,240,20.0,2,1,1,0.351665,0.154478,0.557358,0.272518,...,2.0,0.314394,-1.109349,-0.572468,1.222410,1.134094,-0.235063,272317.0,0.121457,1
80680,2739,1,253,21.1,1,1,1,0.531246,0.208078,0.754717,0.354560,...,2.0,0.748088,-0.905436,1.414109,0.032372,2.343126,0.744692,257610.0,0.113166,0
81754,2740,1,232,19.3,2,1,1,0.310943,0.218462,0.460633,0.387235,...,2.0,1.381240,0.986549,-0.969899,0.301802,-0.313214,-0.344655,249019.0,0.128593,1
81903,2749,0,231,19.2,2,1,1,0.432610,0.283153,0.819576,0.509537,...,2.0,0.531327,-0.885234,0.085093,1.994165,-0.831330,-1.902806,208434.0,0.121055,1
81043,2750,0,249,20.8,2,2,1,0.162409,0.096761,0.429102,0.259678,...,2.0,-0.330777,-0.788469,-0.798989,-2.036093,-0.072305,-0.746145,246636.0,0.127136,1


In [7]:
df_node.head()

,,squeakycleanExclude,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,...,mc_190,mc_191,mc_192,mc_193,mc_194,mc_195,mc_196,mc_197,mc_198,mc_199
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,0,-0.163692,-0.194793,-0.738457,-0.660451,-0.507375,-0.942911,-0.771878,-1.043019,-1.004840,...,0.037811,-0.028927,0.011956,0.051684,0.012334,0.016364,-0.000210,0.000552,0.004936,0.005656
80680,2739,1,-0.189720,0.419498,0.255181,0.363531,-0.362685,-0.021325,0.344072,-0.428808,-0.389079,...,0.025966,0.000917,-0.077783,-0.035653,-0.007192,-0.007894,-0.007147,-0.002556,-0.010794,0.000718
81754,2740,1,0.392162,0.492997,0.896959,0.045772,-0.384540,-0.164547,0.711992,0.731543,0.326487,...,0.002225,0.058244,0.034252,0.011500,-0.007661,0.017174,0.021998,0.005729,0.005387,0.010277
81903,2749,0,-0.008190,-0.018243,-0.445943,0.157206,0.242949,0.118706,0.072858,-0.271011,0.125997,...,0.027635,0.055917,-0.008735,0.034511,0.001779,0.009687,0.011967,0.004857,0.004425,0.007941
81043,2750,0,-0.417882,-0.230303,-1.001550,0.225743,-0.179732,-0.175516,-0.195755,-0.253353,0.049064,...,-0.014485,0.051826,-0.030353,0.036327,0.006498,0.014257,0.015699,-0.000378,-0.002993,0.001960


# Prepare files for normative modelling

In [8]:
# Note, 'ageAtScan1_Years' is assumed to be covs[0] and 'sex_adj' is assumed to be covs[1]
# if more than 2 covs are to be used, append to the end and age/sex will be duplicated accordingly in the forward model
covs = ['ageAtScan1', 'sex_adj']

print(covs)
num_covs = len(covs)
print(num_covs)

['ageAtScan1', 'sex_adj']
2


In [9]:
extra_str_2 = ''

## Primary model (train/test split)

In [10]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
normativedir = os.path.join(os.environ['MODELDIR'], '+'.join(covs) + extra_str_2 + '/')
print(normativedir)
if not os.path.exists(normativedir): os.mkdir(normativedir);

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_200_streamlineCount/ageAtScan1+sex_adj/


In [11]:
# Write out training -- retaining only residuals from nuissance regression
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'train.csv'))
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_train = df_node[df_node[train_test_str] == 0].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_train), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_train.to_csv(os.path.join(normativedir, 'resp_train.csv'))
resp_train.to_csv(os.path.join(normativedir, 'resp_train.txt'), sep = ' ', index = False, header = False)

# Write out test -- retaining only residuals from nuissance regression
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'test.csv'))
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_test = df_node[df_node[train_test_str] == 1].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_test), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_test.to_csv(os.path.join(normativedir, 'resp_test.csv'))
resp_test.to_csv(os.path.join(normativedir, 'resp_test.txt'), sep = ' ', index = False, header = False)

print(str(resp_train.shape[1]) + ' features written out for normative modeling')

1000 features written out for normative modeling


### Forward variants

In [12]:
fwddir = os.path.join(normativedir, 'forward/')
if not os.path.exists(fwddir): os.mkdir(fwddir)

# Synthetic cov data
x = get_synth_cov(df, cov = 'ageAtScan1', stp = 12)

if 'sex_adj' in covs:
    # Produce gender dummy variable for one repeat --> i.e., to account for two runs of ages, one per gender
    gender_synth = np.concatenate((np.ones(x.shape),np.zeros(x.shape)), axis = 0)

# concat
synth_cov = np.concatenate((np.matlib.repmat(x, 2, 1), np.matlib.repmat(gender_synth, 1, 1)), axis = 1)
print(synth_cov.shape)

# write out
np.savetxt(os.path.join(fwddir, 'synth_cov_test.txt'), synth_cov, delimiter = ' ', fmt = ['%.1f', '%.d'])

(30, 2)


### Cross-val variant

In [13]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
cvdir = os.path.join(normativedir, 'cv/')
if not os.path.exists(cvdir): os.mkdir(cvdir)

### Permutation test | train and test | no blocks

In [14]:
# # number of permutations
# num_perms = 1000

# # Set seed for reproducibility
# np.random.seed(0)

# for i in range(num_perms):
#     permdir = os.path.join(normativedir, 'perm_all/perm_' + str(i))
#     if not os.path.exists(permdir): os.makedirs(permdir)

#     df_shuffed = df.copy()
#     df_shuffed.loc[:,covs] = df_shuffed[covs].sample(frac = 1).values
#     df_shuffed.loc[:,covs[1]] = df_shuffed.loc[:,covs[1]].astype(int)

#     df_shuffed[df_shuffed[train_test_str] == 0].to_csv(os.path.join(permdir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)
#     df_shuffed[df_shuffed[train_test_str] == 1].to_csv(os.path.join(permdir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)